In [60]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from PIL import Image

plt.style.use('ggplot')

using 3 different methods
- EasyOCR method
- TesseractOCR method
- PaddleOCR method

In [49]:
annot = pd.read_parquet('C:/Users/Admin/OneDrive/Desktop/ImageToText/dataset/annot.parquet')
imgs = pd.read_parquet('C:/Users/Admin/OneDrive/Desktop/ImageToText/dataset/img.parquet')
img_fns = glob('C:/Users/Admin/OneDrive/Desktop/ImageToText/dataset/train_val_images/train_images/*')

In [ ]:
annot.head()

In [ ]:
imgs.head()

In [ ]:
#C:\Users\Admin\OneDrive\Desktop\ImageToText\dataset\train_val_images\train_images\00a0db6495982c1d.jpg
img_id = img_fns[0].split('\\')[-1].split('.')[0]

fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(plt.imread(img_fns[0]))
plt.show()

In [ ]:
plt.imshow(plt.imread(img_fns[0]))

In [ ]:
image_id = img_fns[0].split('\\')[-1].split('.')[0]
print(annot.head())
print(annot['image_id'].unique())  # Display unique values for comparison
result = annot.query('image_id == @image_id')
print(result)

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(20,20))
axs = axs.flatten()
for i in range(25):
    axs[i].imshow(plt.imread(img_fns[i]))
    axs[i].axis('off')
    image_id = img_fns[i].split('\\')[-1].rstrip('.jpg')
    n_annot = len(annot.query('image_id == @image_id'))
    axs[i].set_title(f'{image_id} - {n_annot}')
plt.show()

Method 1. pytesseract 

In [ ]:
!pip install tesseract-ocr
!pip install pytesseract easyocr opencv-python -q
!pip install paddleocr paddlepaddle-gpu -q

In [ ]:
import pytesseract
#pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

print(pytesseract.image_to_string(img_fns[6], lang='eng'))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(plt.imread(img_fns[6]))
ax.axis('off')
plt.show()

Method 2. easyocr

In [ ]:
import easyocr
reader = easyocr.Reader(['en'], gpu = True)

In [97]:
results = reader.readtext(img_fns[6])

In [ ]:
pd.DataFrame(results, columns=['bbox','text','conf'])

Method 3. keras_ocr

In [ ]:
pip install keras-ocr --upgrade

In [104]:
!pip install tensorflow -q

In [ ]:
import keras_ocr
pipeline = keras_ocr.pipeline.Pipeline()

In [ ]:
results = pipeline.recognize([img_fns[11]])
pd.DataFrame(results[0], columns=['text', 'bbox'])

In [ ]:
keras_ocr.tools.drawAnnotations(plt.imread(img_fns[11], results[0]))

In [ ]:
# Easyocr
reader = easyocr.Reader(['en'], gpu = False)

dfs = []
for img in tqdm(img_fns[:25]):
    result = reader.readtext(img)
    img_id = img.split('\\')[-1].split('.')[0]
    img_df = pd.DataFrame(result, columns=['bbox', 'text', 'conf'])
    img_df['img_id'] = img_id
    dfs.append(img_df)
easyocr_df = pd.concat(dfs)

In [ ]:
%%time
# keras_ocr
pipeline = keras_ocr.pipeline.Pipeline()
dfs = []
for img in tqdm(img_fns[:25]):
    results = pipeline.recognize([img])
    result = results[0]
    img_id = img.split('\\')[-1].split('.')[0]
    img_df = pd.DataFrame(result, columns=['text', 'bbox'])
    img_df['img_id'] = img_id
    dfs.append(img_df)
kesarocr_df = pd.concat(dfs)

In [ ]:
def plot_compare(img_fn, easyocr_df, kerasocr_df):
    img_id = img_fn.split('/')[-1].split('.')[0]
    fig, axs = plt.subplots(1, 2, figsize=(15, 10))

    easy_results = easyocr_df.query('img_id == @img_id')[['text','bbox']].values.tolist()
    easy_results = [(x[0], np.array(x[1])) for x in easy_results]
    keras_ocr.tools.drawAnnotations(plt.imread(img_fn), 
                                    easy_results, ax=axs[0])
    axs[0].set_title('easyocr results', fontsize=24)

    keras_results = kerasocr_df.query('img_id == @img_id')[['text','bbox']].values.tolist()
    keras_results = [(x[0], np.array(x[1])) for x in keras_results]
    keras_ocr.tools.drawAnnotations(plt.imread(img_fn), 
                                    keras_results, ax=axs[1])
    axs[1].set_title('keras_ocr results', fontsize=24)
    plt.show()

In [ ]:
# Loop over results
for img_fn in img_fns[:25]:
    plot_compare(img_fn, easyocr_df, kerasocr_df)

In [ ]:
keras_ocr.tools.drawAnnotations(plt.imread(img_fns[11], results[0], ax=ax))